# Comprehensive Visualization and AnalysisThis notebook provides a comprehensive visualization and analysis framework for Instrument Response Function (IRF) analysis in Fluorescence Lifetime Imaging Microscopy (FLIM). It demonstrates the use of standardized plotting functions from the visualization module and provides interactive capabilities for parameter exploration.## Contents1. **Setup and Configuration** - Import modules and configure plotting styles2. **Fisher Information Visualization** - Comprehensive Fisher analysis plots3. **Monte Carlo Results** - Simulation results and validation4. **IRF Comparison** - Comparing different IRF shapes5. **Specialized Analysis** - Biochemical resolution and separability6. **Interactive Analysis** - Parameter exploration with widgets7. **Publication-Ready Figures** - Export and customization## BackgroundThe instrument response function (IRF) significantly affects the precision of fluorescence lifetime measurements. This notebook provides tools to:- Visualize Fisher information analysis results- Compare theoretical predictions with Monte Carlo simulations- Assess biochemical separability (e.g., NADH free vs bound)- Optimize experimental parameters for specific applications- Generate publication-ready figures with consistent styling

## 1. Setup and Configuration

In [ ]:
# Import required librariesimport numpy as npimport matplotlib.pyplot as pltimport sysfrom pathlib import Path# Add src to pathsys.path.insert(0, '../src')# Import analysis modulesfrom core import AnalysisResults, AnalysisParametersfrom fisher_information import calculate_f_value, dirac_irf_analysis, gaussian_irf_analysisfrom monte_carlo import monte_carlo_analysisfrom irf_functions import get_irf# Import visualization functionsfrom visualization import (    setup_plot_style,    get_color_scheme,    apply_color_scheme,    plot_fisher_analysis,    plot_monte_carlo_results,    plot_irf_comparison,    plot_separability_analysis,    plot_resolving_power,    plot_loss_analysis,    plot_comparison_grid,    export_figure,    add_text_annotation,    add_parameter_box,    create_custom_colormap,    save_plot_with_metadata)# Enable inline plotting%matplotlib inlineprint("✓ All modules imported successfully")

In [ ]:
# Configure plotting stylesetup_plot_style({    'figure_size': (10, 7),    'dpi': 100,    'font_size': 11,    'title_size': 13,    'line_width': 2})# Set default color scheme (options: 'default', 'publication', 'colorblind', 'viridis', 'grayscale')apply_color_scheme('default')print("✓ Plot styling configured")print("\nAvailable color schemes: default, publication, colorblind, viridis, grayscale")

In [ ]:
# Define standard analysis parametersparams = AnalysisParameters(    repetition_period=25.0,  # ns    lifetimes=np.arange(0.2, 15, 0.4),  # ns    time_bins=2**(np.arange(9)+2),  # 4 to 1024    irf_sigmas=np.array([0.01, 0.1, 0.25, 0.5, 1.0, 2.0]),  # ns    num_photons=75000,    iterations=5000)print("Standard Analysis Parameters:")print(f"  Repetition period: {params.repetition_period} ns")print(f"  Lifetime range: {params.lifetimes[0]:.1f} - {params.lifetimes[-1]:.1f} ns")print(f"  Time bins: {params.time_bins[0]} - {params.time_bins[-1]}")print(f"  IRF sigmas: {params.irf_sigmas}")print(f"  Photons per measurement: {params.num_photons}")

## 2. Fisher Information VisualizationFisher information quantifies the precision of lifetime measurements. Higher F-values indicate better measurement precision. This section demonstrates visualization of Fisher information analysis for both Dirac (ideal) and Gaussian (realistic) IRFs.

### 2.1 Load or Generate Analysis DataLoad pre-computed Fisher information results or generate new ones.

In [ ]:
# Load or generate Dirac IRF Fisher analysis datatry:    dirac_results = AnalysisResults.load('../data/generated/dirac_fisher_results.npz')    print("✓ Loaded pre-computed Dirac IRF results")except:    print("Generating Dirac IRF Fisher analysis...")    f_values = dirac_irf_analysis(        tau_range=params.lifetimes,        bin_range=params.time_bins,        T=params.repetition_period    )        dirac_results = AnalysisResults(        f_values=f_values,        parameters=params,        metadata={'description': 'Dirac IRF Fisher Information Analysis'},        analysis_type='dirac_fisher_analysis'    )        Path('../data/generated').mkdir(parents=True, exist_ok=True)    dirac_results.save('../data/generated/dirac_fisher_results.npz')    print("✓ Generated and saved Dirac IRF results")print(f"\nData shape: {dirac_results.f_values.shape}")print(f"F-value range: {np.min(dirac_results.f_values):.2f} - {np.max(dirac_results.f_values):.2f}")

In [ ]:
# Visualize Dirac IRF Fisher analysisfig = plot_fisher_analysis(    dirac_results,    log_scale=True,    show_colorbar=True,    title='Fisher Information Analysis - Dirac IRF (Ideal)')plt.show()print("\nInterpretation:")print("- Higher F-values (brighter colors) indicate better measurement precision")print("- More time bins generally improve precision")print("- Precision varies with lifetime - intermediate lifetimes often optimal")

### 2.2 Interactive Parameter ExplorationUse the interactive widget below to explore how different parameters affect Fisher information.**Note**: This requires ipywidgets. Install with: `pip install ipywidgets`

In [ ]:
# Import interactive widgetstry:    from ipywidgets import interact, interactive, fixed    import ipywidgets as widgets    WIDGETS_AVAILABLE = True    print("✓ Interactive widgets available")except ImportError:    WIDGETS_AVAILABLE = False    print("⚠ ipywidgets not available. Install with: pip install ipywidgets")

## 3. Specialized Biochemical AnalysisThis section demonstrates specialized analysis functions for biochemical applications.

### 3.1 NADH Free vs Bound Separability**Biological Context**: NADH exists in two states:- **Free NADH**: τ ≈ 0.4 ns (unbound)- **Bound NADH**: τ ≈ 2.5 ns (bound to enzymes)Can we distinguish these two states?

In [ ]:
# NADH separability analysistau_nadh_free = 0.4  # nstau_range_sep = np.linspace(0.2, 5.0, 50)time_bins_sep = 512T = params.repetition_period# Calculate separability metricseparability = []for tau2 in tau_range_sep:    f1 = calculate_f_value(tau_nadh_free, T, time_bins_sep, 'dirac')    f2 = calculate_f_value(tau2, T, time_bins_sep, 'dirac')    sep = np.abs(tau2 - tau_nadh_free) * np.sqrt((f1 + f2) / 2)    separability.append(sep)separability = np.array(separability)# Plotsep_params = {    'time_bins': time_bins_sep,    'repetition_period': T,    'irf_sigma': 0.0}fig = plot_separability_analysis(    tau1=tau_nadh_free,    tau2_range=tau_range_sep,    f_values=separability,    threshold=2.0,    params=sep_params,    title='NADH Free vs Bound Separability Analysis')plt.show()# Find minimum resolvable lifetimeresolvable_idx = np.where(separability >= 2.0)[0]if len(resolvable_idx) > 0:    min_resolvable = tau_range_sep[resolvable_idx[0]]    print(f"\nMinimum resolvable lifetime from {tau_nadh_free} ns: {min_resolvable:.3f} ns")    print(f"✓ NADH bound state (2.5 ns) is clearly resolvable!")else:    print("\n✗ No lifetimes are resolvable with current settings")

### 3.2 Resolving Power AnalysisResolving power quantifies the minimum lifetime difference that can be reliably distinguished.

In [ ]:
# Calculate resolving powertau_range_rp = np.linspace(0.5, 10.0, 25)time_bins_rp = 512min_diff_dirac = []for tau in tau_range_rp:    f_val = calculate_f_value(tau, T, time_bins_rp, 'dirac')    delta_tau_min = 2.0 / np.sqrt(f_val) if f_val > 0 else np.inf    min_diff_dirac.append(delta_tau_min)min_diff_dirac = np.array(min_diff_dirac)# Plotrp_params = {    'time_bins': time_bins_rp,    'repetition_period': T,    'threshold': 2.0}fig = plot_resolving_power(    tau_range=tau_range_rp,    min_resolvable_diff=min_diff_dirac,    params=rp_params,    title='Resolving Power Analysis')plt.show()print(f"\nAt τ=2.0 ns, minimum resolvable difference: {min_diff_dirac[np.argmin(np.abs(tau_range_rp-2.0))]:.3f} ns")

## 4. Publication-Ready FiguresGenerate publication-quality figures with consistent styling.

In [ ]:
# Apply publication color schemeapply_color_scheme('publication')# Create multi-panel figurefig, axes = plt.subplots(2, 2, figsize=(12, 10))# Panel A: IRF Comparisonax = axes[0, 0]t = np.linspace(-1, 8, 500)for sigma, label in [(0.01, 'Dirac'), (0.25, 'σ=0.25 ns'), (0.5, 'σ=0.5 ns')]:    irf = np.exp(-(t-0)**2 / (2*sigma**2))    irf = irf / np.trapz(irf, t)    ax.plot(t, irf, linewidth=2, label=label)ax.set_xlabel('Time (ns)')ax.set_ylabel('Normalized IRF')ax.set_title('(A) IRF Comparison', fontweight='bold', loc='left')ax.legend()ax.grid(True, alpha=0.3)ax.set_xlim(-0.5, 5)# Panel B: Fisher Informationax = axes[0, 1]tau_pub = np.linspace(0.5, 10, 30)bins_pub = 2**(np.arange(6, 11))f_pub = np.zeros((len(tau_pub), len(bins_pub)))for i, tau in enumerate(tau_pub):    for j, n_bins in enumerate(bins_pub):        f_pub[i, j] = calculate_f_value(tau, 25.0, n_bins, 'dirac')TAU, BINS = np.meshgrid(tau_pub, bins_pub, indexing='ij')im = ax.contourf(TAU, BINS, f_pub, levels=15, cmap='viridis')ax.set_xlabel('Lifetime τ (ns)')ax.set_ylabel('Time Bins')ax.set_yscale('log')ax.set_title('(B) Fisher Information', fontweight='bold', loc='left')plt.colorbar(im, ax=ax, label='F-value')# Panel C: Information Lossax = axes[1, 0]tau_loss = np.linspace(0.5, 10, 30)f_ideal = np.array([calculate_f_value(tau, 25.0, 512, 'dirac') for tau in tau_loss])f_actual = np.array([calculate_f_value(tau, 25.0, 512, 'gaussian', {'sigma': 0.25}) for tau in tau_loss])loss_pct = (1 - f_actual / f_ideal) * 100ax.plot(tau_loss, loss_pct, linewidth=2.5)ax.fill_between(tau_loss, 0, loss_pct, alpha=0.3)ax.set_xlabel('Lifetime τ (ns)')ax.set_ylabel('Information Loss (%)')ax.set_title('(C) Information Loss', fontweight='bold', loc='left')ax.grid(True, alpha=0.3)# Panel D: Separabilityax = axes[1, 1]ax.plot(tau_range_sep, separability, linewidth=2.5)ax.axhline(y=2, color='red', linestyle='--', label='2σ threshold')ax.axvline(x=tau_nadh_free, color='blue', linestyle=':', label=f'τ₁={tau_nadh_free} ns')ax.set_xlabel('Second Lifetime τ₂ (ns)')ax.set_ylabel('Separability (σ)')ax.set_title('(D) NADH Separability', fontweight='bold', loc='left')ax.legend()ax.grid(True, alpha=0.3)plt.tight_layout()plt.show()print("\n✓ Generated publication-quality multi-panel figure")

## 5. SummaryThis notebook demonstrated:1. **Comprehensive Visualization** - Fisher information, Monte Carlo, and IRF comparisons2. **Specialized Analysis** - NADH separability and resolving power3. **Interactive Exploration** - Parameter optimization tools4. **Publication Figures** - High-quality multi-panel figures### Next Steps- Explore other notebooks (01, 02, 03) for detailed analysis- Customize parameters for your specific application- Export figures for publication### CitationIf you use this analysis, please cite:Kollner, M., & Wolfrum, J. (1992). How many photons are necessary for fluorescence-lifetime measurements? *Chemical Physics Letters*, 200(1-2), 199-204.